In [10]:
import numpy as np
from collections import defaultdict 

In [8]:
def split2int(s):
    return list(map(int, s.split()))

In [21]:
data = 'kittens.in'
end2cache_latency = defaultdict(dict)
with open('data/{}'.format(data), 'r') as f:
    V, E, R, C, X = split2int(f.readline())
    video_sizes = {i: int(size)
                   for i, size in enumerate(f.readline().strip().split())}
    for endpoint_i in range(E):
        latency2center, n_caches = split2int(f.readline())
        for _ in range(n_caches):
            cache, latency = split2int(f.readline())
            end2cache_latency[endpoint_i][cache] = latency            
        endpoint_i += 1
    

In [19]:
end2cache_latency[0][499]

85